In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
! pip install google-cloud-discoveryengine

In [ ]:
from typing import List, Optional
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1beta as discoveryengine


Set Google Cloud project information and data store ID.
To run this notebook, you must have an existing Google Cloud project and enable the Agent Builder API.

If you don't know your project ID, try the following:

Run gcloud config list.
Run gcloud projects list.
See the support page: Locate the project ID
You can also change the LOCATION variable for your Vertex Search data store if needed. 

In [ ]:

project_id = "your-project-id"
location = "global"  # Options: "global", "us", "eu" default to "global"
data_store_id = "your-datastore-id-"

1. Function Definition:
**def search_sample(...)**: This line defines the function named search_sample.
**project_id: str**: This parameter takes the project ID of your Google Cloud project as a string.
**location: str**: This parameter takes the location of your data store as a string (e.g., "global", "us", "eu").
**data_store_id: str**: This parameter takes the ID of your data store as a string.
**search_query: str:** This parameter takes the search query you want to execute as a string.
f**ilter_query = None**: This parameter is optional and takes a filter query as a string. It allows you to refine your search results.
**-> List[discoveryengine.SearchResponse]**: This indicates that the function returns a list of SearchResponse objects from the Discovery Engine API.

In [ ]:
def search_sample(
    project_id: str,
    location: str,
    data_store_id: str,
    search_query: str,
    filter_query = None
) -> List[discoveryengine.SearchResponse]:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    
    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=1,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
        filter=filter_query
    )
    response = client.search(request)
    print(response)
    return response

2. Client Configuration:
**client_options**: This section sets up the client options for connecting to the Discovery Engine API. It dynamically constructs the API endpoint based on the specified location.
**client = discoveryengine.SearchServiceClient(...)**: This line creates a client object to interact with the Discovery Engine API using the configured client options.
**serving_config**: This line retrieves the full resource name of the serving configuration for your data store. This configuration defines how search requests are processed.

In [ ]:
client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)
    # The full resource name of the search engine serving config
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        serving_config="default_config",
    )


3. Content Search Specification:
**content_search_spec**: This section defines the specific content search options for your request. It includes:
**snippet_spec**: Specifies whether to return snippets from the search results.
**extractive_content_spec**: Specifies options for extracting relevant content from the search results.
**summary_spec:** Specifies options for generating summaries of the search results.

In [ ]:
# Optional: Configuration options for search
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_answer_count=1,
            max_extractive_segment_count=1,
            return_extractive_segment_score=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=1,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
        ),
    )

4. Search Request:
**request**: This line constructs a SearchRequest object that encapsulates all the parameters for your search operation. It includes:
**serving_config**: The serving configuration to use.
**query**: The search query.
**page_size**: The maximum number of results to return (set to 1 in this case).
**content_search_spec**: The content search specifications.
**query_expansion_spec**: Specifies whether to automatically expand the search query.
**spell_correction_spec**: Specifies whether to automatically correct spelling errors in the search query.
**filter**: The optional filter query.


In [ ]:
# Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=1,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
        filter=filter_query
    )
    response = client.search(request)
    print(response)
    return response

**run it !** 
feel free to replace **search_query** with other questions related to the data store documents. 

In [ ]:

response = search_sample(
    project_id=project_id,
    location=location,
    data_store_id=data_store_id,
    search_query="What was YouTube revenue in the second quarter of 2020?",
)
response